In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [7]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [8]:
data = requests.get(team_urls[0])

In [9]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [10]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [11]:
data = requests.get(f"https://fbref.com{links[0]}")

In [12]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [13]:
shooting.head()

For Tottenham Hotspur                                                  \
                   Date   Time            Comp         Round  Day Venue   
0            2023-08-13  14:00  Premier League   Matchweek 1  Sun  Away   
1            2023-08-19  17:30  Premier League   Matchweek 2  Sat  Home   
2            2023-08-26  12:30  Premier League   Matchweek 3  Sat  Away   
3            2023-08-29  19:45         EFL Cup  Second round  Tue  Away   
4            2023-09-02  15:00  Premier League   Matchweek 4  Sat  Away   

                                        ... Standard               Expected  \
  Result     GF     GA        Opponent  ...     Dist   FK PK PKatt       xG   
0      D      2      2       Brentford  ...     19.6  0.0  0     0      1.3   
1      W      2      0  Manchester Utd  ...     13.8  0.0  0     0      1.7   
2      W      2      0     Bournemouth  ...     16.6  1.0  0     0      2.1   
3      D  1 (3)  1 (5)          Fulham  ...      NaN  NaN  0     0      NaN   
4      W      5      2         Burnley  ...     19.3  0.0  0     0      2.2   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  1.3    0.07  0.7     0.7        Match Report  
1  1.7    0.10 -0.7    -0.7        Match Report  
2  2.1    0.12 -0.1    -0.1        Match Report  
3  NaN     NaN  NaN     NaN        Match Report  
4  2.2    0.11  2.8     2.8        Match Report  

[5 rows x 26 columns]

In [17]:
shooting.columns = shooting.columns.droplevel()

ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [18]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-13,14:00,Premier League,Matchweek 1,Sun,Away,D,2,2,Brentford,...,4-2-3-1,Robert Jones,Match Report,NaN,18,6,19.6,0.0,0,0
1,2023-08-19,17:30,Premier League,Matchweek 2,Sat,Home,W,2,0,Manchester Utd,...,4-2-3-1,Michael Oliver,Match Report,NaN,17,6,13.8,0.0,0,0
2,2023-08-26,12:30,Premier League,Matchweek 3,Sat,Away,W,2,0,Bournemouth,...,4-2-3-1,Tim Robinson,Match Report,NaN,17,6,16.6,1.0,0,0
3,2023-08-29,19:45,EFL Cup,Second round,Tue,Away,D,1 (3),1 (5),Fulham,...,4-4-1-1,Joshua Smith,Match Report,Fulham won on penalty kicks following extra time,10,3,NaN,NaN,0,0
4,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Away,W,5,2,Burnley,...,4-2-3-1,Darren England,Match Report,NaN,21,11,19.3,0.0,0,0


In [19]:
years = list(range(2022, 2020, -1))
all_matches = []

In [20]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [21]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [22]:
len(all_matches)

40

In [23]:
match_df = pd.concat(all_matches)

In [24]:
match_df.columns = [c.lower() for c in match_df.columns]

In [25]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-13,14:00,Premier League,Matchweek 1,Sun,Away,D,2,2,Brentford,...,Match Report,NaN,18.0,6.0,19.6,0.0,0,0,2022,Tottenham Hotspur
1,2023-08-19,17:30,Premier League,Matchweek 2,Sat,Home,W,2,0,Manchester Utd,...,Match Report,NaN,17.0,6.0,13.8,0.0,0,0,2022,Tottenham Hotspur
2,2023-08-26,12:30,Premier League,Matchweek 3,Sat,Away,W,2,0,Bournemouth,...,Match Report,NaN,17.0,6.0,16.6,1.0,0,0,2022,Tottenham Hotspur
4,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Away,W,5,2,Burnley,...,Match Report,NaN,21.0,11.0,19.3,0.0,0,0,2022,Tottenham Hotspur
5,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Home,W,2,1,Sheffield Utd,...,Match Report,NaN,28.0,10.0,16.4,0.0,0,0,2022,Tottenham Hotspur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,...,Match Report,NaN,4.0,3.0,17.3,0.0,0,0,2021,Southampton
43,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,...,Match Report,NaN,18.0,4.0,14.0,0.0,1,1,2021,Southampton
44,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Home,L,0,2,Fulham,...,Match Report,NaN,5.0,1.0,24.2,0.0,0,0,2021,Southampton
45,2023-05-21,14:00,Premier League,Matchweek 37,Sun,Away,L,1,3,Brighton,...,Match Report,NaN,5.0,1.0,13.8,1.0,0,0,2021,Southampton


In [26]:
match_df.to_csv("matches.csv")